# <p style='margin-bottom:-30px'><font color='darkblue'>Modul 7: Übungsmaterial Assoziationsanalyse &#8211; Und was hörst du?</font>&nbsp; <font size='6'>&#127911;</font></p>

<hr style="border:1px solid gray"> </hr>

Im zweiten Video dieses Moduls haben Sie die Begriffe Konfidenz und Support kennengelernt. Im Rahmen der Assoziationsanalyse tauchen auch noch andere Begriffe wie Lift, Leverage und Conviction auf. Diese sollen in diesem Notebook erklärt werden. Anschließend wird mit der Assoziationsanalyse ein Datensatz mit Musikdaten analysiert.

## <font color='darkblue'>Inhalt</font>


1. [Weitere Kennwerte der Assoziationsanalyse](#kap1)  
    1.1 [Wiederholung der Warenkorbanalyse](#kap11)  
    1.2 [Lift](#kap12)  
    1.3 [Leverage und Conviction](#kap13)
    

2. [Anwendung der Assoziationsanalyse auf einen Musik-Datensatz](#kap2)    
    2.1 [Datensatz einlesen und erforschen](#kap21)    
    2.2 [Datensatz vorbereiten](#kap22)   
    2.3 [Modellierung](#kap23)  
    2.4 [Erweiterung um die Kenwerte der Assoziationsanalyse](#kap24)


3. [Fazit](#kap3)

<hr style="border:1px solid gray"> </hr>

## <font color='darkblue'>1. Weitere Kennwerte der Assoziationsanalyse</font> <a name="kap1"></a>

### <font color='darkblue'>1.1 Wiederholung der Warenkorbanalyse</font> <a name="kap11"></a>

Zunächst werden noch einmal die Schritte bis zu den Assoziationsregeln aus dem anderen Notebook durchgeführt:

In [ ]:
import pandas as pd
from mlxtend.preprocessing import TransactionEncoder

In [ ]:
dataset = [['Apfel', 'Avocado', 'Chips'],
           ['Apfel', 'Avocado', 'Chips', 'Bier', 'Wasser'],
           ['Avocado', 'Chips', 'Bier'],
           ['Bier', 'Wasser'],
           ['Avocado', 'Chips', 'Wasser'],
           ['Apfel', 'Avocado', 'Wasser'],
           ['Apfel', 'Avocado', 'Chips'],
           ['Apfel', 'Avocado', 'Chips']
          ]

In [ ]:
te = TransactionEncoder()
te_matrix = te.fit(dataset).transform(dataset)
df = pd.DataFrame(te_matrix, columns=te.columns_)
print(df)

In [ ]:
from mlxtend.frequent_patterns import apriori
frequent_itemsets = apriori(df, min_support=0.5, use_colnames=True)

frequent_itemsets

In [ ]:
from mlxtend.frequent_patterns import association_rules

rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.8)
rules

Hier sind jetzt die Begriffe Lift, Leverage und Conviction zu sehen. Sie werden nun erklärt:

### <font color='darkblue'>1.2 Lift</font> <a name="kap12"></a>

Der Lift ist eine weitere Größe, die im Rahmen der Assoziationsanalyse genutzt wird und in der vorstehenden Tabelle zu finden ist. Er berechnet sich durch

$$\operatorname{lift}(A\Rightarrow B) = \frac{\text{Anzahl der Einkäufe von $A$ und $B$}}{\text{Anzahl der Einkäufe von $A$}\cdot\text{Anzahl der Einkäufe von $B$}}$$

d.h. die Anzahl der gemeinsamen Einkäufe von $A$ und $B$ wird in Relation zu den Einkäufen von $A$ und den Einkäufen von $B$ (und nicht wie bei der Konfidenz nur zu den Einkäufen von $A$) gesetzt. Dadurch wird verhindert, dass eine Scheinregel in Betracht gezogen wird, die dadurch entsteht, dass $B$ in sehr vielen Einkäufen vorkommt.

Der Lift kann folgende Werte annehmen:
- $\operatorname{lift}(A\Rightarrow B) = 1$ bedeutet, dass $A$ und $B$ unabhängig voneinander gekauft werden
- $\operatorname{lift}(A\Rightarrow B) > 1$ bedeutet, dass $B$ wahrscheinlich gekauft wird, wenn $A$ gekauft wird
- $\operatorname{lift}(A\Rightarrow B) < 1$ bedeutet, dass $B$ wahrscheinlich *nicht* gekauft wird, wenn $A$ gekauft wird

Die Tabelle zeigt, dass alle Liftwerte über 1 liegen, was bedeutet, dass hier keine Scheinregeln vorliegen, sondern die Waren tatsächlich abhängig voneinander gekauft werden. Aufgrund des kleinen Datensatzes liegen die Werte von Lift aber nur wenig über 1, in Anwendungen werden größere Werte erwartet!

Durch Umformungen folgt:

$$\operatorname{lift}(A\Rightarrow B) = \frac{\operatorname{conf}(A\Rightarrow B)}{\operatorname{supp}(B)}$$

Hieraus kann die folgende Interpretation gezogen werden: Der Lift von 1,14 bei der ersten Regel Apfel &rarr; Avocado bedeutet konkret, dass Avocados von Apfel-Käufer:innen 14% öfter gekauft werden als vom Durchschnitt aller Käufer:innen. 

Mit `mlextend` kann sowohl bei den gefundenen Regeln der zugehörige Lift ausgegeben werden, als auch bei der Erzeugung/Berechnung der Regeln statt auf die Konfidenz auf den Lift geachtet werden:

<div class="alert alert-block alert-success">
&#128187; <b>Arbeitsauftrag:</b> 

Setzen Sie die folgenden Parameter so fest, dass die Metrik `lift` mit einem Grenzwert von `1.1` ausgewählt wird.
</div>

In [ ]:
# Durchführung der Assoziationsanalyse mit Lift
# rules = association_rules(frequent_itemsets, metric=#, min_threshold=#)
rules

Es sollten sich in diesem Fall 6 Assoziationsregeln ergeben. 

### <font color='darkblue'>1.3 Leverage und Conviction</font> <a name="kap13"></a>

Zwei weitere Kennwerte bei der Assoziationsanalyse sind Leverage und Conviction. 

**Leverage** - zu deutsch Hebel - ist definiert durch

$$\operatorname{leverage}(A\Rightarrow B) = \operatorname{supp}(A\Rightarrow B) - \operatorname{supp}(A) \cdot \operatorname{supp}(B)
$$

Der Leverage ist in seiner Aussagekraft stark mit dem Lift vergleichbar: 

- $\operatorname{leverage}(A\Rightarrow B) = 0$ bedeutet, dass $A$ und $B$ unabhängig voneinander gekauft werden
- $\operatorname{leverage}(A\Rightarrow B) > 0$ bedeutet, dass $B$ wahrscheinlich gekauft wird, wenn $A$ gekauft wird
- $\operatorname{leverage}(A\Rightarrow B) < 0$ bedeutet, dass $B$ wahrscheinlich *nicht* gekauft wird, wenn $A$ gekauft 

Die sehr kleinen Leverage Werte im Beispiel liegen an der Größe des Datensatzes (bei sehr wenigen Artikeln ist ein gemeinsamer Kauf schneller Zufall als bei vielen Artikeln). 

**Conviction** - zu deutsch Überzeugungswert - ist definiert als

$$\operatorname{conviction}(A\Rightarrow B) = \frac{1- \operatorname{supp}(B)}{1 - \operatorname{conf}(A\Rightarrow B) }
$$

Die Conviction kann Werte von 0 bis unendlich annehmen. Ein hoher Wert zeigt an, dass der Kauf von B sehr abhängig ist von A. Bei einer Konfidenz von 1, d.h. alle Käufer:innen von A kauften auch B, wird der Nenner 0 und Conviciton wird unendlich groß.

Conviction kann dabei wie folgt interpretiert werden: das Verhältnis der erwarteten Häufigkeit des Auftretens von A ohne B (d. h. der Häufigkeit, mit der die Regel eine falsche Vorhersage trifft), wenn A und B unabhängig wären, dividiert durch die beobachtete Häufigkeit falscher Vorhersagen. Im Beispiel zeigt der Überzeugungswert von 1,25, dass die Regel "$Apfel \Rightarrow Chips$" 1,25 mal so häufig falsch liegen würde, wenn der Zusammenhang zwischen Apfel und Chips rein zufällig wäre. 
Im Beispiel zeigen sich auch drei Einträge mit einem `Inf` bei conviction. Dies ist der Fall, da IMMER, wenn ein Apfel bzw. Chips oder Äpfel und Chips gekauft wurden, auch Avocados gekauft wurden. 

<hr style="border:1px solid gray"> </hr>

## <font color='darkblue'>2. Anwendung der Assoziationsanalyse auf einen Musik-Datensatz</font> <a name="kap2"></a>

Die Assoziationsanalyse wird nicht nur zur Analyse von Warenkörben benutzt. Sie kann auch auf den hier vorgestellten Datensatz angewendet werden. Aus einer Liste, in der erfasst ist, welche:r Hörer:in, welche Musik gehört hat, können Regeln für Musik, die oft zusammen gehört wird, erstellt werden. Daraus lassen sich dann leicht Empfehlungen ableiten. 

### <font color='darkblue'>2.1 Datensatz einlesen und erforschen</font> <a name="kap21"></a>

Der Datensatz stammt ursprünglich von <a href="https://www.kaggle.com/ravichaubey1506/lastfm">Kaggle</a> und enthält Musikdaten aus dem Jahr 2011. Der Datenstaz wird eingelesen und die ersten Informationen ausgegeben: 

In [ ]:
df = pd.read_csv("M7_Uebung_Musikdaten.csv")
df.info()

Der Datensatzt enthält keine leeren Felder. Außer der Spalte `user` sind alle Spalten als object-Datentyp abgespeichert.  
Die Datenspalten sind in der folgenden Tabelle beschrieben.

<table align='left'>
    <thead>
        <tr>
          <th style="text-align:left">Spaltenname</th>
          <th style="text-align:left">Bedeutung</th>
          <th style="text-align:left">Weitere Informationen</th>
        </tr>
    </thead>
    <tr>
    <td style="text-align:left">user</td>
    <td style="text-align:left">User ID</td>
    <td style="text-align:left">-</td>
    </tr>
    <tr>
    <td style="text-align:left">artist</td>
    <td style="text-align:left">Name von Band oder Künstler:in</td>
    <td style="text-align:left">-</td>
    </tr>
    <tr>
    <td style="text-align:left">sex</td>
    <td style="text-align:left">Geschlecht User</td>
    <td style="text-align:left">m (male = männlich), f (female = weiblich)</td>
    </tr>
    <tr>
    <td style="text-align:left">country</td>
        <td style="text-align:left">Land User</td>
    <td style="text-align:left">-</td>
    </tr>
</table>

Zuerst wird der Datensatz nun in mehreren Arbeitsaufträgen genauer analysiert:

<div class="alert alert-block alert-success">
&#128187; <b>Arbeitsauftrag:</b> 

Bestimmen Sie die Anzahl der verschiedenen vorkommenden Bands und Künstler:innen, sowie die Anzahl der Hörer:innen im Datensatz. 
    
Tipp: Kombinieren Sie den Befehl `unique()` mit einem weiteren Befehl.  
</div>

In [ ]:
# Bestimmung der Anzahl an Bands und Künstler:innen 

In [ ]:
# Bestimmung der Anzahl an Anzahl der Hörer:innen

Der Datensatz enthält 1004 Bands und Künstler:innen  und 15.000 Hörer:innen. 

<div class="alert alert-block alert-success">
&#128187; <b>Arbeitsauftrag:</b> 

Bestimmen Sie nun die am häufigsten genannte Bands und Künstler:innen und die am seltensten genannte Bands und Künstler:innen 
    
Tipp: Hierbei ist der Befehl `value_counts` sehr hilfreich. 
</div>


In [ ]:
# Bestimmung der am häufigsten genannten Anzahl an Bands und Künstler:innen 

In [ ]:
# Bestimmung der am seltensten genannten Anzahl an Bands und Künstler:innen 

*Radiohead* und *The Beatles* sind in diesem Datensatz am häufigsten vertreten.

### <font color='darkblue'>2.2 Datensatz vorbereiten</font> <a name="kap22"></a>

Für die Anwendung des Apriori Algorithmus muss der Datensatz noch vorbereitet werden. Im Notebook zum Video war der Datensatz eine lange Liste, bei der jeder Eintrag einem Einkauf entsprach. 

Ein Einkauf entspricht hier einer hörenden Person. Allerdings enthielt ein Einkauf alle Teile, die eingekauft wurden. Dementsprechend muss nun für jede hörende Person sortiert werden, welche Bands und Künstler:innen gehört werden. Diese Liste soll `transactions` heißen. 

Das Anlegen geschieht mit dem folgenden Befehl: 

In [ ]:
# Erklärung: für jede hörende Person werden alle Bands und Künstler:innen in einer Liste gesammelt und diese
# als Element an transaction drangehängt
transactions = []
for i in df['user'].unique():   
    transactions.append(list(df[df['user'] == i]['artist'].values))  
    # Bilde eine Liste über alle Bands und Künstler:innen, die gehört wurden, und hänge sie an transactions an

Es entsteht eine sehr große Variable, auf die im nächsten Kapitel der Apriori Algorithmus angewendet werden kann.  

In [ ]:
transactions

### <font color='darkblue'>2.3 Modellierung</font> <a name="kap23"></a>

Der Apriori Algorithmus wird nun eingebunden und mit `min_support=0.007`und `min_confidence=0.7` durchgeführt. Die Größenordnung dieser Werte unterscheidet sich sehr stark vom ersten Beispiel, da es einen deutlich größeren "Warenkorb" gibt.

In [ ]:
from efficient_apriori import apriori
itemsets, rules = apriori(transactions, min_support=0.007,  min_confidence=0.7)

In [ ]:
len(rules)

In [ ]:
rules

Diese Kombination an Parametern gibt zufälligerweise genau eine Regel aus, die inhaltlich auch plausibel erscheint.

<div class="alert alert-block alert-success">
&#128187; <b>Arbeitsauftrag:</b> 

Verändern Sie  `min_support` oder `min_confidence` so, dass mehr Regeln ausgegeben werden. Müssen die Parameter dazu größer oder kleiner gemacht werden?

</div>

In [ ]:
# Platz für den Arbeitsauftrag 
# itemsets_neu, rules_neu = apriori(transactions, min_support=#,  min_confidence=#)
display(len(rules_neu))
rules_neu

Eine kleinere Konfidenz und ein kleinerer Mindest-Support bewirkt mehr Regeln. Inhaltlch scheinen die entstehenden Regeln plausibel zu sein. 

### <font color='darkblue'>2.4 Erweiterung um die Kennwerte der Assoziationsanalyse</font> <a name="kap24"></a>

Es wird nun das folgende Modell weiter untersucht: 

In [ ]:
itemsets, rules = apriori(transactions, min_support=0.007,  min_confidence=0.6) # löschen
rules

Nun sollen die bekannten Kennwerte (Support, Confidence, Lift und Conviction) zu den entstandenen Regeln ausgegeben werden. 

Da das Paket `mlxtend` so große Datenmengen nicht verarbeiten kann, soll in diesem Abschnitt eine ähnliche Tabelle mit dem Paket `efficient_apriori` erstellt werden. Leverage wird dabei ausgelassen, da der Befehl hierfür sehr umfassend wäre (Wenn Sie eine aufwändige Programmieraufgabe ausprobieren wollen: Dies ist eine!)

<div class="alert alert-block alert-success">
&#128187; <b>Arbeitsauftrag:</b> 

Wenden Sie nachfolgend die Methode zur Ausgabe von Support (Name: `support`), Confidence (Name: `confidence`), Lift (Name: `lift`) und Conviction (Name: `conviction`) auf die Variable `rules[1]` an. 
</div>

In [ ]:
# Ausgabe des Supports 
rules[1]

In [ ]:
# Ausgabe der Konfidenz
rules[1]

In [ ]:
# Ausgabe des Lifts 
rules[1]

In [ ]:
# Ausgabe der Conviction
rules[1]

Die Methoden `lhs` bzw. `rhs` geben "A" und "B" der Assoziationsregeln aus: 

In [ ]:
print(rules[1].lhs)
print(rules[1].rhs)

Werden alle diese Methoden kombiniert, so kann daraus die gewünschte Tabelle zusammengebaut werden: 

In [ ]:
def regeln(rules):
    eintrag = []
    for x in rules:
        eintrag.append([x.lhs, x.rhs, x.support, x.confidence, x.lift, x.conviction])
    return pd.DataFrame(eintrag, columns=["antecedents", "consequents", "support","confidence","lift", "conviction"])

In [ ]:
ergebnis = regeln(rules)
ergebnis

Folgendes kann für die Werte festgestellt werden: 
- Der Support liegt im Gegensatz zu dem kleinen Warenkorb am Anfang nun im kleinen Prozentbereich. Er ist wie folgt zu intepretieren: Bei der ersten Regel hörten 2,2% aller Höher:innen *Keane* und *Coldplay*. 
- Die Konfidenzwerte liegen alle bei etwa 0,6. Die erste Regel lässt sich so interpretieren: Wurde *Keane* gehört, wurde in 64% der Fälle auch *Coldplay* gehört.
- Der Lift liegt nun für alle Regeln bei mindestens 3. Der Lift von 4,0 bei der Regel *Keane* &rarr; *Coldplay* bedeutet beispielsweise, dass *Coldplay* von *Keane*-Hörer:innen vier Mal so oft gehört wird wie der Durchschnitt aller Hörer:innen. 
- Auch die Conviction liegt nun mindestens bei 2. Dies ist nicht besonders groß, wenn bedacht wird, dass Conviction unendlich groß werden kann. Grund dafür ist die Größenordnung der Konfidenzwerte. 

<div class="alert alert-block alert-success">
&#128187; <b>Arbeitsauftrag:</b> 

Sortieren Sie die Variable `ergebnis` nach den Ausprägungen der Varable `lift`.
    
Tipp: Nutzen Sie die Methode `sort_values()`
</div>

In [ ]:
# Platz für Arbeitsauftrag

Falls der Arbeitsauftrag geklappt hat, sollte nun an oberster Stelle die Assoziationsregel "(beyoncé, britney spears)	&rarr; rihanna" stehen. Auch diese Regel ist sehr gut nachvollziehbar.

<hr style="border:1px solid gray"> </hr>

## <font color='darkblue'>3. Fazit</font> <a name="kap3"></a>

Dieses Notebook hat: 
- neue Begriffe der Assoziationsanalyse vorgestellt und ihre Bedeutung erläutert. 
- den Apriori Algorithmus auf einen Musikdatensatz angewendet. 